## Live audio

In [4]:
import azure.cognitiveservices.speech as speechsdk

AZURE_SPEECH_KEY = 'e5404bd89ea14c388c2c17234f95e36a'
AZURE_SERVICE_REGION = 'southeastasia'

# Initialize configurations
speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SERVICE_REGION)

# Set the LanguageIdMode to Continuous
speech_config.set_property(property_id=speechsdk.PropertyId.SpeechServiceConnection_LanguageIdMode, value='Continuous')

conv_speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SERVICE_REGION)

# Auto language detection configuration
auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(languages=["en-SG", "zh-CN", "ms-MY", "hi-IN"])

def speech_recognizer_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def speech_recognizer_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def speech_recognizer_recognized_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED_SR:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f'\tText={evt.result.text}')
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print('\tText={}'.format(evt.result.text))
        print('\tSpeaker ID={}'.format(evt.result.speaker_id))
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')        

def speech_recognizer_recognizing_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    if evt.result.reason == speechsdk.ResultReason.RecognizingSpeech:
        print(f'RECOGNIZING: {evt.result.text}')

def speech_recognizer_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')

# Initialize recognizer
audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
speech_recognizer = speechsdk.SpeechRecognizer(
    speech_config=speech_config,
    auto_detect_source_language_config=auto_detect_source_language_config,
    audio_config=audio_config
)

conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
    speech_config=speech_config,
    audio_config=audio_config,
    auto_detect_source_language_config=auto_detect_source_language_config
)
transcribing_stop = False

def stop_cb(evt: speechsdk.SessionEventArgs):
    # Callback to stop continuous recognition upon receiving an event `evt`
    print('CLOSING on {}'.format(evt))
    global transcribing_stop
    transcribing_stop = True

# Connect callbacks
speech_recognizer.recognized.connect(speech_recognizer_recognized_cb)
speech_recognizer.recognizing.connect(speech_recognizer_recognizing_cb)
speech_recognizer.session_started.connect(speech_recognizer_session_started_cb)
speech_recognizer.session_stopped.connect(speech_recognizer_session_stopped_cb)
speech_recognizer.canceled.connect(speech_recognizer_recognition_canceled_cb)

conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)


# Start continuous recognition
print("Starting continuous recognition...")
speech_recognizer.start_continuous_recognition_async()
conversation_transcriber.start_transcribing_async()

transcribing_stop = False



# Keep the program running to listen to the microphone
import time
try:
    while True:
        time.sleep(0.1)
           
except KeyboardInterrupt:
    print("KeyBorad Stopping continuous recognition...")
    speech_recognizer.stop_continuous_recognition_async()
    #conversation_transcriber.session_stopped.connect(stop_cb)
    #conversation_transcriber.canceled.connect(stop_cb)
    #conversation_transcriber.stop_transcribing_async() 
    print("Recognition stopped.")

Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.


Exception ignored in: <function _Handle.__del__ at 0x7fb0c84248b0>
Traceback (most recent call last):
  File "/Users/lihuicham/opt/anaconda3/envs/speechsdk/lib/python3.9/site-packages/azure/cognitiveservices/speech/interop.py", line 106, in __del__
    self.__release_fn(self.__handle)
KeyboardInterrupt: 


Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
Starting continuous recognition...
SessionStarted event
SessionStarted event


## Using File

In [1]:
import azure.cognitiveservices.speech as speechsdk

AZURE_SPEECH_KEY = 'e5404bd89ea14c388c2c17234f95e36a'
AZURE_SERVICE_REGION = 'southeastasia'
AUDIO_FILE_PATH = "/Users/lihuicham/Documents/GitHub/speechsdk/data/nsc/3010_short.wav"  # Replace with your actual audio file path

# Initialize configurations
speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SERVICE_REGION)

# Set the LanguageIdMode to Continuous
speech_config.set_property(property_id=speechsdk.PropertyId.SpeechServiceConnection_LanguageIdMode, value='Continuous')
speech_config.set_property(property_id=speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults, value='true')

# Auto language detection configuration
auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(languages=["en-SG", "zh-CN", "ms-MY", "hi-IN"])

### SPEECH RECOGNIZER CALLBACKS ###

def speech_recognizer_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def speech_recognizer_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def speech_recognizer_recognized_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED_SR:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f'\tText={evt.result.text}')
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

def speech_recognizer_recognizing_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    if evt.result.reason == speechsdk.ResultReason.RecognizingSpeech:
        print(f'RECOGNIZING: {evt.result.text}')

def speech_recognizer_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')

### CONVERSATION TRANSCRIBER CALLBACKS ###

def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print('\tText={}'.format(evt.result.text))
        print('\tSpeaker ID={}'.format(evt.result.speaker_id))
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

def conversation_transcriber_transcribing_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBING:')
    print('\tText={}'.format(evt.result.text))
    print('\tSpeaker ID={}'.format(evt.result.speaker_id))

def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')        



# Initialize recognizer using an audio file instead of the microphone
audio_config = speechsdk.audio.AudioConfig(filename=AUDIO_FILE_PATH)
speech_recognizer = speechsdk.SpeechRecognizer(
    speech_config=speech_config,
    auto_detect_source_language_config=auto_detect_source_language_config,
    audio_config=audio_config
)

conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
    speech_config=speech_config,
    audio_config=audio_config,
    auto_detect_source_language_config=auto_detect_source_language_config
)
transcribing_stop = False

def stop_cb(evt: speechsdk.SessionEventArgs):
    # Callback to stop continuous recognition upon receiving an event `evt`
    print('CLOSING on {}'.format(evt))
    global transcribing_stop
    transcribing_stop = True

# Connect callbacks
# speech_recognizer.recognized.connect(speech_recognizer_recognized_cb)
# speech_recognizer.recognizing.connect(speech_recognizer_recognizing_cb)
# speech_recognizer.session_started.connect(speech_recognizer_session_started_cb)
# speech_recognizer.session_stopped.connect(speech_recognizer_session_stopped_cb)
# speech_recognizer.canceled.connect(speech_recognizer_recognition_canceled_cb)

conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
conversation_transcriber.transcribing.connect(conversation_transcriber_transcribing_cb)
conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)


# Start continuous recognition from an audio file
print(f"Starting continuous recognition from {AUDIO_FILE_PATH}...")
speech_recognizer.start_continuous_recognition_async()
conversation_transcriber.start_transcribing_async()

# Keep the program running to process the audio file
import time
try:
    while not transcribing_stop:
        time.sleep(0.1)
except KeyboardInterrupt:
    print("Stopping recognition...")
    speech_recognizer.stop_continuous_recognition_async()
    conversation_transcriber.stop_transcribing_async()
    print("Recognition stopped.")

Starting continuous recognition from /Users/lihuicham/Documents/GitHub/speechsdk/data/nsc/3010_short.wav...
SessionStarted event
TRANSCRIBING:
	Text=ya so uh we
	Speaker ID=Unknown
TRANSCRIBING:
	Text=ya so uh we just start
	Speaker ID=Unknown
TRANSCRIBING:
	Text=ya so uh we just start talking
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so umm we
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so umm we should we start with maybe
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so umm we should we start with maybe the 1st
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so umm we should we start with maybe the first one
	Speaker ID=Guest-1
TRANSCRIBING:
	Text=ya so uh we just start talking right OK so umm we should we start with maybe the first one or
	Speaker ID=Guest-1
TRANS

TRANSCRIBING:
	Text=33AH
	Speaker ID=Unknown
TRANSCRIBED:
	Text=It's cheap, ah.
	Speaker ID=Guest-2
TRANSCRIBED:
	Text=So less than $100,000 in sing dollars divided by.
	Speaker ID=Guest-1
Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
TRANSCRIBED:
	Text=33AH.
	Speaker ID=Guest-1
SessionStopped event
Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: received close frame, sending a close response frame.
Info: on_underlying_io_close_sent: uws_client=0x7fbc9a0a2870, io_send_result:0
Info: on_underlying_io_close_sent: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
Info: on_underlying_io_bytes_received: Close frame r